<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#概要" data-toc-modified-id="概要-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>概要</a></span></li><li><span><a href="#2019-01-04" data-toc-modified-id="2019-01-04-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>2019-01-04</a></span><ul class="toc-item"><li><span><a href="#Two-Sigma:気になったカーネル" data-toc-modified-id="Two-Sigma:気になったカーネル-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Two Sigma:気になったカーネル</a></span><ul class="toc-item"><li><span><a href="#Create-Data-Pipeline-and-LGBClassifierCV(score:0.65811)" data-toc-modified-id="Create-Data-Pipeline-and-LGBClassifierCV(score:0.65811)-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Create Data Pipeline and LGBClassifierCV(score:0.65811)</a></span></li></ul></li></ul></li><li><span><a href="#2019-01-08" data-toc-modified-id="2019-01-08-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>2019-01-08</a></span><ul class="toc-item"><li><span><a href="#Two-Sigma-参考になったカーネル" data-toc-modified-id="Two-Sigma-参考になったカーネル-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Two Sigma 参考になったカーネル</a></span></li><li><span><a href="#アイディア" data-toc-modified-id="アイディア-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>アイディア</a></span><ul class="toc-item"><li><span><a href="#性能が出る期間を探してマルチモデル構築" data-toc-modified-id="性能が出る期間を探してマルチモデル構築-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>性能が出る期間を探してマルチモデル構築</a></span></li></ul></li></ul></li></ul></div>

# 概要
Kaggleに関連する調査研究を書き溜めてゆくノートである。

# 2019-01-04
## Two Sigma:気になったカーネル

### Create Data Pipeline and LGBClassifierCV(score:0.65811)
以下のカーネルが気になっている。  
https://www.kaggle.com/aditya1702/create-data-pipeline-and-lgbclassifiercv  

これまで公開されてきたカーネルの要点を消化して、よくまとめているカーネルである。  
注目点は、  
* きちんとクラスを作っている
* 指数平滑移動平均など時系列特徴量(time-lagged-feature)の追加  
* パイプライン処理の導入
   ```python
        data_pipeline = Pipeline([
            ('quant_features_generator', AddingQuantFeatures()),
            ('news_preprocessor', PreprocessingNews()),
            ('news_unstacker', UnstackingNews()),
            ('news_grouping', GroupingNewsWithAggregations()),
            ('lgb', LGBClassifierCV(cv = 5,
                                    perform_random_search = False,
                                    perform_bayes_search = False,
                                    use_train_test_split = False,
                                    use_full_data = True,
                                    use_kfold_split = False)
            )
        ])

        data_pipeline.fit([market_train, news_train])
    ```

* かなり簡素なツリー構造。詳細はこちら。
  ```python
            self.lgb_optimal_params = {'learning_rate': 0.05, 
                                       'num_leaves': 12, 
                                       'n_estimators': 100, 
                                       'min_child_samples': 70, 
                                       'colsample_bytree': 0.9, 
                                       'subsample': 0.9, 
                                       'reg_alpha': 0.8, 
                                       'reg_lambda': 0.2, 
                                       'n_jobs': 3}
  
  ```
    
    * num_leaves が 12 とかなり極端に小さな値になっている。

* Post-Process で新しいパターンを発見した。  
  平均値を引いている。どんな分布になるのだろうか？
  ```python
          preds = pipeline.predict([market_obs_df, news_obs_df])
          preds = (preds - np.mean(preds))
  ```

# 2019-01-08
## Two Sigma 参考になったカーネル
https://www.kaggle.com/felsal/combined-of-market-data-only-new-beginnig

回帰モデルを複数構築して、アンサンブルで予測している。  
0.639のスコアを達成している。  

## アイディア

### 性能が出る期間を探してマルチモデル構築

半年～１年の期間を1ヶ月 or 2週間とかスライドさせながら、モデルを構築し、バリデーション期間に対して性能が良いものを探す。  
そして、上位10件などをモデルとして採用する。

モデルを構築したら、アンサンブルで予測するのである。

シグナルを含む期間を見つけても、既存の期間と混ぜるとかえって性能が悪化することがあった。  
それならばいっそのことモデルを分けてしまったほうが良いのかもしれない。


In [1]:
preds = []
import numpy as np
for _ in range(10):
    preds.append(np.array(np.random.randint(1,10,100)))

In [2]:
preds

[array([3, 1, 2, 4, 1, 6, 7, 4, 4, 6, 4, 7, 9, 9, 1, 9, 7, 6, 5, 6, 3, 4, 4,
        7, 9, 5, 9, 1, 2, 5, 1, 2, 9, 9, 2, 7, 9, 2, 9, 8, 1, 2, 6, 9, 9, 3,
        8, 4, 8, 1, 5, 2, 2, 1, 4, 5, 6, 1, 8, 6, 9, 2, 1, 6, 1, 8, 7, 2, 2,
        5, 5, 2, 8, 9, 5, 7, 9, 9, 5, 8, 4, 4, 5, 9, 2, 9, 6, 8, 6, 4, 3, 6,
        1, 9, 4, 4, 2, 2, 2, 5]),
 array([8, 7, 3, 8, 1, 3, 7, 3, 6, 9, 4, 6, 5, 9, 9, 5, 3, 3, 2, 1, 8, 2, 7,
        7, 7, 9, 6, 2, 2, 7, 5, 8, 8, 2, 8, 3, 2, 6, 8, 7, 1, 3, 8, 5, 1, 3,
        2, 5, 7, 1, 5, 5, 3, 1, 8, 9, 1, 4, 2, 8, 8, 8, 1, 9, 7, 5, 7, 3, 9,
        4, 4, 4, 2, 4, 7, 4, 6, 7, 4, 8, 3, 3, 9, 9, 3, 1, 5, 4, 3, 1, 1, 2,
        9, 2, 5, 8, 1, 5, 3, 4]),
 array([3, 3, 4, 9, 3, 9, 6, 9, 2, 6, 6, 6, 8, 9, 6, 2, 2, 8, 5, 6, 2, 5, 2,
        6, 5, 2, 6, 1, 5, 3, 4, 2, 1, 3, 3, 8, 4, 9, 5, 9, 3, 4, 8, 5, 1, 7,
        8, 1, 3, 8, 6, 6, 6, 5, 2, 9, 4, 1, 1, 1, 9, 7, 7, 9, 4, 8, 6, 8, 9,
        4, 4, 3, 8, 6, 6, 6, 2, 7, 5, 2, 5, 8, 7, 7, 2, 8, 5, 6, 9, 9, 3, 3,
        

In [5]:
total_preds = None
for _preds in preds:
    if total_preds is None:
        total_preds = _preds
    else:
        total_preds += _preds

In [6]:
total_preds

array([53, 34, 43, 58, 36, 51, 61, 54, 42, 52, 59, 48, 53, 48, 56, 51, 41,
       57, 52, 47, 45, 36, 39, 62, 51, 46, 56, 45, 52, 52, 51, 37, 51, 58,
       46, 64, 59, 53, 55, 66, 32, 38, 58, 50, 50, 43, 57, 51, 44, 60, 66,
       41, 51, 32, 50, 51, 53, 46, 62, 38, 69, 51, 37, 55, 55, 57, 54, 46,
       48, 43, 58, 47, 53, 50, 53, 39, 55, 53, 47, 49, 34, 50, 56, 66, 39,
       49, 57, 60, 57, 54, 48, 50, 43, 58, 42, 47, 37, 38, 48, 32])

In [8]:
total_preds/10 * 2 -1

array([  9.6,   5.8,   7.6,  10.6,   6.2,   9.2,  11.2,   9.8,   7.4,
         9.4,  10.8,   8.6,   9.6,   8.6,  10.2,   9.2,   7.2,  10.4,
         9.4,   8.4,   8. ,   6.2,   6.8,  11.4,   9.2,   8.2,  10.2,
         8. ,   9.4,   9.4,   9.2,   6.4,   9.2,  10.6,   8.2,  11.8,
        10.8,   9.6,  10. ,  12.2,   5.4,   6.6,  10.6,   9. ,   9. ,
         7.6,  10.4,   9.2,   7.8,  11. ,  12.2,   7.2,   9.2,   5.4,
         9. ,   9.2,   9.6,   8.2,  11.4,   6.6,  12.8,   9.2,   6.4,
        10. ,  10. ,  10.4,   9.8,   8.2,   8.6,   7.6,  10.6,   8.4,
         9.6,   9. ,   9.6,   6.8,  10. ,   9.6,   8.4,   8.8,   5.8,
         9. ,  10.2,  12.2,   6.8,   8.8,  10.4,  11. ,  10.4,   9.8,
         8.6,   9. ,   7.6,  10.6,   7.4,   8.4,   6.4,   6.6,   8.6,   5.4])